In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
import japanize_matplotlib
from process import * 
import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

target_column="target3"
make_genre_list=False

In [3]:
#augumented_data = pd.read_pickle(data_folda+'feature_added_after2021.pkl')
pseudo_time_series_with = pd.read_pickle(data_folda+'pseudo_time_series_with_v_hotpepper.pkl')

In [4]:
master_data_20240301=pd.read_csv(data_folda+'restaurant_data.csv')

/tmp/ipykernel_23433/3375611667.py:1: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  master_data_20240301=pd.read_csv(data_folda+'restaurant_data.csv')


In [5]:
train_20231130=pseudo_time_series_with[pseudo_time_series_with["base_date"]==pd.datetime(2023,11,30)]

/tmp/ipykernel_23433/1627555179.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  train_20231130=pseudo_time_series_with[pseudo_time_series_with["base_date"]==pd.datetime(2023,11,30)]


In [6]:
train_20231130[train_20231130["eval_period"]==True]["target3"].value_counts()

0    881820
Name: target3, dtype: int64

In [7]:
master_data_20240301["close_date"]=pd.to_datetime(master_data_20240301["close_date"],errors="coerce")

In [8]:
# business_status=0 もしくは、closed_dateが2023年11月30日以降、かつ掲載開始が11月30日以前の店舗の restaurant_id を取得
nonbaias_restaurant_id=master_data_20240301[((master_data_20240301["business_status"]==0)
                     |(master_data_20240301["close_date"]>"2023-11-30"))
                    &(master_data_20240301["created_at"]<="2023-11-30")]["id"]
                     

In [9]:
nonbaias_restaurant_id

2          23037738
3          35008980
4          44005194
5          54002517
10         65000665
             ...   
1668202    17012892
1668481    30002915
1668553    13088729
1668563    33003667
1668870    11017567
Name: id, Length: 837462, dtype: int64

In [10]:
# train_20231130のoriginal_idがnonbaias_restaurant_idに含まれるものを取得
train_20231130=train_20231130[train_20231130["original_id"].isin(nonbaias_restaurant_id)]
# その条件に合致するものは、eval_periodをTrueにする
train_20231130.loc[train_20231130["original_id"].isin(nonbaias_restaurant_id),"eval_period"]=True

In [11]:
train_20231130["closed_flg"].value_counts()

0    821283
1        29
Name: closed_flg, dtype: int64

In [12]:
# master_data_20240301 の中で 2023年12月1日から2024年2月29日に閉店した店舗の restaurant_id を取得
closed_in_3month_from20231201=master_data_20240301[(master_data_20240301["close_date"]>"2023-11-30")
                     &(master_data_20240301["close_date"]<"2024-03-01")
                     &(master_data_20240301["business_status"]==1)]["id"]


In [13]:
# train_20231130のoriginal_idがclosed_in_3month_from20231201に含まれるものについては、target3を1にする
train_20231130.loc[train_20231130["original_id"].isin(closed_in_3month_from20231201),"target3"]=1

In [14]:
train_20231130["eval_period"].value_counts()

True    821312
Name: eval_period, dtype: int64

In [15]:
train_20231130.to_pickle(data_folda+'pseudo_time_series_with_v_hotpepper_non_bias.pkl')

In [16]:
train_20231130

,restaurant_id,original_id,closed_date,business_status,prefecture,base_date,closed_flg,target3,target6,eval_period,...,infos.fanCount,infos.reviewTags.count1,infos.reviewTags.count2,infos.reviewTags.count3,infos.reviewTags.count4,infos.reviewTags.count5,infos.reviewTags.count6,infos.reviewTags.count7,infos.reviewTags.count8,infos.reviewTags.count9
11,00000528-ea7b-4364-bb76-3d46fba520ef,22001886,NaT,掲載中,静岡県,2023-11-30,0,0,0,True,...,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,00003d54-0a2a-43f5-b945-5ae9b0314942,22035496,NaT,掲載中,静岡県,2023-11-30,0,0,0,True,...,311.0,48.0,29.0,3.0,4.0,NaN,1.0,3.0,7.0,NaN
47,00003ef6-4867-409a-896c-566134666192,22013184,NaT,掲載中,静岡県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,00005256-746d-467d-bf65-82686b244108,23061742,NaT,掲載中,愛知県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,00007ebe-24c4-4082-92b1-8cdb1732274d,23055999,NaT,掲載中,愛知県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10810235,ffff88ea-fd34-46f8-a749-001a2c59cdba,22022016,NaT,掲載中,静岡県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10810247,ffff9349-bd6a-4c56-bb06-c1c11af3b736,11036290,NaT,掲載中,埼玉県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10810259,ffffbb36-5a31-4f02-b1e5-5883dfc16f09,40011715,NaT,掲載中,福岡県,2023-11-30,0,0,0,True,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10810271,ffffdb1a-05bf-4227-8ea8-cf5bb0b63a43,47003447,NaT,掲載中,沖縄県,2023-11-30,0,0,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# target =0 , 1で比を計算
# nonbiasのtarget0,1 それぞれのカウント
nonbias_count=train_20231130.groupby("target3").count().T

In [18]:
# target =0 , 1で比を計算
train_20231130.groupby("target3").mean().T

/tmp/ipykernel_23433/136250465.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_20231130.groupby("target3").mean().T


target3,0,1
original_id,2.088038e+07,1.944662e+07
closed_flg,0.000000e+00,7.397959e-03
target6,0.000000e+00,7.397959e-03
eval_period,1.000000e+00,1.000000e+00
id,2.088038e+07,1.944662e+07
latitude,3.551122e+01,3.562225e+01
longitude,1.370319e+02,1.374898e+02
net_reservation_flg,5.528437e-02,9.515306e-02
coupons_count,9.104200e-02,1.772959e-01
reviews_count,2.428336e+01,2.361378e+01


In [19]:
bias_data=pseudo_time_series_with[(pseudo_time_series_with["eval_period"]==True)
                        &(pseudo_time_series_with["base_date"]==pd.datetime(2023,7,31))]

/tmp/ipykernel_23433/1052708399.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  &(pseudo_time_series_with["base_date"]==pd.datetime(2023,7,31))]


In [20]:
bias_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 867424 entries, 7 to 10810279
Data columns (total 85 columns):
 #   Column                            Non-Null Count   Dtype          
---  ------                            --------------   -----          
 0   restaurant_id                     867424 non-null  object         
 1   original_id                       867424 non-null  int64          
 2   closed_date                       5468 non-null    datetime64[ns] 
 3   business_status                   867424 non-null  object         
 4   prefecture                        867424 non-null  object         
 5   base_date                         867424 non-null  datetime64[ns] 
 6   closed_flg                        867424 non-null  int64          
 7   target3                           867424 non-null  int64          
 8   target6                           867424 non-null  int64          
 9   eval_period                       867424 non-null  bool           
 10  id                

In [21]:
bias_data["target3"].value_counts()

0    862833
1      4591
Name: target3, dtype: int64

In [22]:
train_20231130["target3"].value_counts()

0    817392
1      3920
Name: target3, dtype: int64

In [23]:
train_20231130_groupby=train_20231130.groupby("target3").mean().T.rename(columns={0:"target0",1:"target1"})

/tmp/ipykernel_23433/4286232008.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_20231130_groupby=train_20231130.groupby("target3").mean().T.rename(columns={0:"target0",1:"target1"})


In [24]:
bias_data_groupby=bias_data.groupby("target3").mean().T.rename(columns={0:"target0_bias",1:"target1_bias"})

/tmp/ipykernel_23433/1982691537.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bias_data_groupby=bias_data.groupby("target3").mean().T.rename(columns={0:"target0_bias",1:"target1_bias"})


In [25]:
nonbias_with_bias=pd.concat([train_20231130_groupby,bias_data_groupby],axis=1)

In [26]:
nonbias_with_bias["nonbias_ratio"]=nonbias_with_bias["target1"]/nonbias_with_bias["target0"]
nonbias_with_bias["bias_ratio"]=nonbias_with_bias["target1_bias"]/nonbias_with_bias["target0_bias"]

nonbias_with_bias["bias_nonbias_ratio"]=nonbias_with_bias["bias_ratio"]/nonbias_with_bias["nonbias_ratio"]
nonbias_with_bias

target3,target0,target1,target0_bias,target1_bias,nonbias_ratio,bias_ratio,bias_nonbias_ratio
original_id,2.088038e+07,1.944662e+07,2.087689e+07,1.951125e+07,0.931335,0.934586,1.003491
closed_flg,0.000000e+00,7.397959e-03,0.000000e+00,0.000000e+00,inf,NaN,NaN
target6,0.000000e+00,7.397959e-03,1.016419e-03,1.000000e+00,inf,983.846066,0.000000
eval_period,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000
id,2.088038e+07,1.944662e+07,2.087689e+07,1.951125e+07,0.931335,0.934586,1.003491
latitude,3.551122e+01,3.562225e+01,3.550925e+01,3.574544e+01,1.003126,1.006652,1.003514
longitude,1.370319e+02,1.374898e+02,1.370328e+02,1.374666e+02,1.003341,1.003165,0.999825
net_reservation_flg,5.528437e-02,9.515306e-02,5.732859e-02,0.000000e+00,1.721157,0.000000,0.000000
coupons_count,9.104200e-02,1.772959e-01,9.120189e-02,0.000000e+00,1.947408,0.000000,0.000000
reviews_count,2.428336e+01,2.361378e+01,2.343027e+01,2.184165e+01,0.972426,0.932198,0.958631
